# <center>Recupération des données </center>
# <center>et mise en base de données </center>

In [ ]:
# Import des librairies pour le Scrapping
import requests
from bs4 import BeautifulSoup
import lxml.html

___

In [ ]:
# Import pour la gestion de la BDD
import sqlite3   # BDD pour le MOC

___

In [ ]:
import pandas as pd
import string
import re
import sys
DEB_Pages=1
NB_Pages=10#2032
liensBD=[]
BdD="database.db"

___

In [ ]:
from datetime import datetime # Module pour la gestion temporelle
import time

In [ ]:
def create_database(app):
    if not path.exists('./' + BdD):
        db.create_all(app)
        print('Created Database!')

___

In [ ]:
def heure():
    x = datetime.now()
    return x.strftime("%m"),x.strftime("%d"),x.strftime("%H"),x.strftime("%M")

In [ ]:
# Critiques de BD:
# Dans cette partie, on récupére (dans une liste), toutes les BD.
# Cette liste contient les liens des pages de la BD et données #
# la décrivant ; comme l'EAN, un résumé, etc...

for idx in range (DEB_Pages, NB_Pages):
    if (idx%800!=0 or idx==1341):
        dom = lxml.html.fromstring(requests.get('https://www.babelio.com/livres-/bande-dessinee/18?page='+str(idx)).content)
        liensBD+=[x for x in dom.xpath('//a/@href') if '/livres/' in x and ('#c') not in x]
        Heure=heure()
        print(Heure[2],":",Heure[3],"Pages ",str(idx)," sur ",str(NB_Pages))
        time.sleep(9)
    else :
        time.sleep(300)

In [ ]:
dfl = pd.DataFrame(liensBD)
dfl=dfl.rename(columns = {0:'liens'})
# On garde une copie des liens récupérer sur le site babelio.com
# On ne sauvegarde que la partie
dfl.to_csv('CSV_tmp/list.csv',sep="\t")

In [ ]:
dfl=pd.read_csv('CSV_tmp/list.csv',sep="\t")
dfl

In [ ]:
dfl=dfl.drop('Unnamed: 0',axis=1) # On retire la colonne inutile

In [ ]:
dfl

In [ ]:
liensBD=dfl['liens']

In [ ]:
def Changer_type(l, dtype=str):
    return list(map(dtype, l))


def remove_tags(text) :
    TAG_RE=re.sub('\n |\t|\n', '', str(text))
    return TAG_RE

In [ ]:
idxsiblocage=0

In [ ]:
# POUR LE POC ON SE CONCENTRE SUR L'AUTEUR PRINCIPAL
# CETTE PARTIE DE CODE ALIMENTE SEULEUMENT LA PARTIE LIVRE ET SA DESCRIPTION
connecteur=sqlite3.connect(BdD)
curseur=connecteur.cursor()
Heure=heure()
print("Début du scrapping: ",Heure[1]," ",Heure[0]," à ",Heure[2],":",Heure[3])
for idx,url_BD in enumerate(liensBD[idxsiblocage:len(liensBD)]):
    
        try:
            idx+=1
            LinksList=[]
            Heure=heure()
            addr="https://www.babelio.com/"+url_BD
            print("Le ",Heure[1]," ",Heure[0]," à ",Heure[2],":",Heure[3],"https://www.babelio.com/"+url_BD,
                  "|| album numéro ",str(idx) , ": ",str(url_BD))
            response_BD=requests.post(addr)
            soup = BeautifulSoup(response_BD.text)
            EAN=soup.find('div',class_="livre_refs grey_light").text
            #print(EAN)
            Date = EAN[70:102][ EAN[70:102].find( '(' )+1 : EAN[70:102].find( ')' ) ]
            EAN=EAN[9:23]
            
            Title=soup.find("h1", itemprop="name").text
            Title=remove_tags(Title)
            #print(Title)
            Image=soup.find("div",class_="col col-4")
            img_src = Image.find('img').attrs['src']
            Auteur1=soup.find("span", itemprop="name").text   
            #print(Auteur1)
            Dessinateur=""
            Description=[des for des in (soup.find("div", itemprop="description",class_="livre_resume")) if des!= '[]']
            Description=remove_tags(Description)
            Genres=[genre.text for genre in (soup.find_all("a",class_="tag_t14"))]
            #print(Description)  
            Rating=soup.find("span", itemprop="ratingValue").text               
            #print(Rating)
            time.sleep(8)
            #print("Sauvegarde en BDD de ",url_BD)
            # Eviter les doublons?
            curseur.execute("Select titre from bd where  titre = ?",[Title]) #Il faut une liste pour éviter l'erreur de Binding
            if len( curseur.fetchall() ) == 0:
                curseur.execute("INSERT INTO bd (titre,ean,genres,resume,auteur,image,date,rating) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",(Title, EAN, str(Genres), Description, Auteur1, img_src, Date, Rating))
                connecteur.commit()
        except Exception as e:
                if e==TimeoutError:
                    print("""Erreur de connection\n il suffit de relancer cette 
                                        section, le script garde en mémoire l'emplacement 
                                        lors du problème.""")
                    print("Sur le livre ",url_BD)
                    print(idx)
                    idxsiblocage=idx
                    break
                else :
                    print("Erreur...",e,"\n")
                    print("Sur le livre ",url_BD)
    
connecteur.close()
print("Fin du scrapping: ",Heure[1]," ",Heure[0]," à ",Heure[2],":",Heure[3])

___
___

___

In [ ]:
idxsiblocagecritique=0

In [24]:
connecteur=sqlite3.connect(BdD)
curseur=connecteur.cursor()
Heure=heure()

print("Début du scrapping: ",Heure[1]," ",Heure[0]," à ",Heure[2],":",Heure[3])
for idx,url_BD in enumerate(liensBD[idxsiblocagecritique:len(liensBD)]):
        Heure=heure()
        #print("Le ",Heure[1]," ",Here[0]," à ",Heure[2],":",Heure[3],"|| Scrap des critiques de: ",str(url_BD))
    
        nbPageCri=1    # Nombres de page de critiques
        maxPageCritique=16                  # On ne gère ici que 15 pages, qu'il y en ait plus
                                            # moins
        while nbPageCri<maxPageCritique:
            try:
                tmp="https://www.babelio.com"+str(url_BD)+"/critiques?a=a&pageN="+str(nbPageCri)
                response_BD=requests.post(tmp)
                soup= BeautifulSoup(response_BD.text)
                Date=[date.text for date in  soup.find_all("span", style=re.compile('"Open Sans",sans-serif' ))]
                #print(Date)
                Critiques=[remove_tags(critique.text) for critique in  soup.find_all("div", id=re.compile('^cri_'))]
                #print(len(Critiques))
                Title=soup.find("h1", itemprop="name").text
                Title=remove_tags(Title)
                #print(Title)
                
                Lst_Lecteurs=[lecteur.text for lecteur in soup.find_all("span", itemprop="name")]
                #print("Nombre de lecteurs: ",len(Lst_Lecteurs))
                Lst_Notes=soup.find_all(itemprop={"ratingValue","content"})
                Lst_Notes=([float(s) for s in re.findall(r'-?\d+\.?\d*', str(Lst_Notes))])
                
                
                #print(len(Lst_Notes),len(Lst_Lecteurs))
                if len(Lst_Notes)!=len(Lst_Lecteurs):
                    del Lst_Notes[0]  # On supprime le 1er élément qui correspond à la note moyenne de la BD
                #print("Nombre de notes: ",len(Lst_Notes))
                
             
                for num,Note in enumerate(Lst_Notes): # On a une critique par lecteur
                    curseur.execute("SELECT * from user where first_name=?",([Lst_Lecteurs[num]]))
                    if len( curseur.fetchall() ) == 0:
                        curseur.execute("INSERT INTO  user (email,password,first_name) VALUES (?,?,?)",(str(Lst_Lecteurs[num])+"BD.BD","12345678",str(Lst_Lecteurs[num])))
                        connecteur.commit()
                    curseur.execute("SELECT titre FROM note where  titre = ? AND auteur= ?",(Title,Lst_Lecteurs[num])) #Il faut une liste pour éviter l'erreur de Binding
                    if len( curseur.fetchall() ) == 0:
                        curseur.execute("INSERT INTO  note (date,titre,note,data,auteur) VALUES (?,?,?,?,?)",(Date[num],Title, Note, Critiques[num],Lst_Lecteurs[num]))
                        connecteur.commit()
            except Exception as e:
                if e==TimeoutError:
                    print("""Erreur de connection\n il suffit de relancer cette 
                         section, le script garde en mémoire l'emplacement 
                         lors du problème.""")
                    idxsiblocagecritique=idx
                    break
                else :
                    print("Erreur...",e,"\n")
                    print("Sur le livre ",url_BD)                    
                
            nbPageCri+=1

# On ferme les données en base
connecteur.close()
print("Fin du scrapping: ",Heure[1]," ",Heure[0]," à ",Heure[2],":",Heure[3])           

Début du scrapping:  20   10  à  14 : 15
Fin du scrapping:  20   10  à  15 : 23
